In [0]:
import networkx as nx
from functools import partial
import pandas as pd
import numpy as np
import pickle
import _pickle as cPickle
#from networkit import linkprediction as lp, nxadapter
#lp for link-prediction, nxadapter for converting nx graph to networkit graph

import csv
import json
from networkx.readwrite import json_graph

from sklearn.metrics.pairwise import cosine_similarity
import tqdm

In [0]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

In [0]:
test_dict_df = pd.read_csv('./input/test_dict_df.csv')
train_dict_df = pd.read_csv('./input/train_dict_df.csv')

test_dict = test_dict_df.to_dict()
train_dict = train_dict_df.to_dict()

In [0]:
with open('./input/embeddings.pickle', 'rb') as handle:
  embeddings = pickle.load(handle)

In [0]:
valid_graph = nx.read_gpickle("./input/valid_graph.gpickle")
test_graph = nx.read_gpickle("./input/test_graph.gpickle")
train_graph = nx.read_gpickle("./input/train_graph.gpickle")

In [9]:
test_dict_df

,src,tar,label,DED,IND,DED_LOG,IND_LOG
0,0,2,0,0.125,0.333333,0.259930,0.366204
1,0,7,0,0.125,0.666667,0.259930,0.732408
2,0,10,0,0.375,0.000000,0.779791,0.000000
3,0,26,0,0.625,0.666667,1.299651,0.732408
4,0,43,0,0.250,0.333333,0.519860,0.366204
5,0,57,0,0.125,0.333333,0.259930,0.366204
6,0,70,0,0.125,0.000000,0.259930,0.000000
7,0,76,0,0.250,0.666667,0.519860,0.732408
8,0,83,0,0.125,0.000000,0.259930,0.000000
9,0,87,0,0.625,0.666667,1.299651,0.732408
